In [1]:
source("../Code/importa_dados.R")

## Ativação dos pacotes necessários:

In [2]:
require(tidyverse, quietly = T)
require(data.table)
require(arules)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Loading required package: arules
Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand


Attaching package: ‘arules’

The following object is masked from ‘package:dplyr’:

    recode

The following objects are masked from ‘package:base’:

    abbreviate, write



## Clientes

In [3]:
glimpse(customers)

Observations: 99,441
Variables: 5
$ customer_id              <chr> "06b8999e2fba1a1fbc88172c00ba8bc7", "18955e8…
$ customer_unique_id       <chr> "861eff4711a542e4b93843c6dd7febb0", "290c77b…
$ customer_zip_code_prefix <int> 14409, 9790, 1151, 8775, 13056, 89254, 4534,…
$ customer_city            <chr> "franca", "sao bernardo do campo", "sao paul…
$ customer_state           <chr> "SP", "SP", "SP", "SP", "SP", "SC", "SP", "M…


In [4]:
customers %>% 
  count(customer_id, sort = T) %>% 
  filter(n > 1) %>% 
  head()

customer_id,n
<chr>,<int>


In [5]:
customers %>% 
  count(customer_unique_id, sort = T) %>% 
  filter(n > 1) %>% 
  head()

customer_unique_id,n
<chr>,<int>
8d50f5eadf50201ccdcedfb9e2ac8455,17
3e43e6105506432c953e165fb2acf44c,9
1b6c7548a2a1f9037c1fd3ddfed95f33,7
6469f99c1f9dfae7733b25662e7f1782,7
ca77025e7201e3b30c44b472ff346268,7
12f5d6e1cbf93dafd9dcc19095df0b3d,6


In [6]:
customers %>% 
  count(customer_state, sort = T) %>% 
  head()

customer_state,n
<chr>,<int>
SP,41746
RJ,12852
MG,11635
RS,5466
PR,5045
SC,3637


## Produtos

In [7]:
glimpse(products)

Observations: 32,951
Variables: 9
$ product_id                 <chr> "1e9e8ef04dbcff4541ed26657ea517e5", "3aa07…
$ product_category_name      <chr> "perfumaria", "artes", "esporte_lazer", "b…
$ product_name_lenght        <int> 40, 44, 46, 27, 37, 60, 56, 56, 57, 36, 54…
$ product_description_lenght <int> 287, 276, 250, 261, 402, 745, 1272, 184, 1…
$ product_photos_qty         <int> 1, 1, 1, 1, 4, 1, 4, 2, 1, 1, 1, 4, 3, 2, …
$ product_weight_g           <int> 225, 1000, 154, 371, 625, 200, 18350, 900,…
$ product_length_cm          <int> 16, 30, 18, 26, 20, 38, 70, 40, 27, 17, 16…
$ product_height_cm          <int> 10, 18, 9, 4, 17, 5, 24, 8, 13, 10, 10, 19…
$ product_width_cm           <int> 14, 20, 15, 26, 13, 11, 44, 40, 17, 12, 16…


In [9]:
products %>% 
  count(product_category_name, sort = T) %>% 
  head()

product_category_name,n
<chr>,<int>
cama_mesa_banho,3029
esporte_lazer,2867
moveis_decoracao,2657
beleza_saude,2444
utilidades_domesticas,2335
automotivo,1900


## Pedidos

In [12]:
glimpse(orders)

Observations: 99,441
Variables: 8
$ order_id                      <chr> "e481f51cbdc54678b7cc49136f2d6af7", "53…
$ customer_id                   <chr> "9ef432eb6251297304e76186b10a928d", "b0…
$ order_status                  <chr> "delivered", "delivered", "delivered", …
$ order_purchase_timestamp      <chr> "2017-10-02 10:56:33", "2018-07-24 20:4…
$ order_approved_at             <chr> "2017-10-02 11:07:15", "2018-07-26 03:2…
$ order_delivered_carrier_date  <chr> "2017-10-04 19:55:00", "2018-07-26 14:3…
$ order_delivered_customer_date <chr> "2017-10-10 21:25:13", "2018-08-07 15:2…
$ order_estimated_delivery_date <chr> "2017-10-18 00:00:00", "2018-08-13 00:0…


In [11]:
orders %>% 
  count(order_status, sort = T)

order_status,n
<chr>,<int>
delivered,96478
shipped,1107
canceled,625
unavailable,609
invoiced,314
processing,301
created,5
approved,2


In [13]:
orders %>% 
  mutate_at(vars(4:8), lubridate::as_datetime) %>% 
  mutate(tempo_aprovacao_entrega = difftime(order_delivered_customer_date, order_approved_at, units = "days") %>% 
           as.numeric() %>% 
           floor(),
         tempo_estimado_realizado = difftime(order_estimated_delivery_date, order_delivered_customer_date, units = "days") %>% 
           as.numeric() %>% 
           floor()) %>% 
  as_tibble() %>% 
  summarise(Tempo_medio_aprovacao_entrega = mean(tempo_aprovacao_entrega, na.rm = T),
            Tempo_medio_estimado_realizado = mean(tempo_estimado_realizado, na.rm = T))

Tempo_medio_aprovacao_entrega,Tempo_medio_estimado_realizado
<dbl>,<dbl>
11.64297,10.87688


## Pagamento

In [6]:
summary(payments$payment_value)

payments %>% 
  count(payment_type, sort = T) %>% 
  head()


    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
    0.00    56.79   100.00   154.10   171.84 13664.08 

payment_type,n
<chr>,<int>
credit_card,76795
boleto,19784
voucher,5775
debit_card,1529
not_defined,3


## Reviews

In [7]:
glimpse(reviews) 

Observations: 100,000
Variables: 7
$ review_id               <chr> "7bc2406110b926393aa56f80a40eba40", "80e641a1…
$ order_id                <chr> "73fc7af87114b39712e6da79b0a377eb", "a548910a…
$ review_score            <int> 4, 5, 5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 4, 4, 3, …
$ review_comment_title    <chr> "", "", "", "", "", "", "", "", "", "recomend…
$ review_comment_message  <chr> "", "", "", "Recebi bem antes do prazo estipu…
$ review_creation_date    <chr> "2018-01-18 00:00:00", "2018-03-10 00:00:00",…
$ review_answer_timestamp <chr> "2018-01-18 21:46:59", "2018-03-11 03:05:13",…


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   4.000   5.000   4.071   5.000   5.000 

review_score,n
<int>,<int>
5,57420
4,19200
1,11858
3,8287
2,3235


In [14]:
summary(reviews$review_score)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   4.000   5.000   4.071   5.000   5.000 

In [15]:
reviews %>% 
  count(review_score, sort = T) %>% 
  head()

review_score,n
<int>,<int>
5,57420
4,19200
1,11858
3,8287
2,3235


## Vendedores

In [8]:
glimpse(sellers)

Observations: 3,095
Variables: 4
$ seller_id              <chr> "3442f8959a84dea7ee197c632cb2df15", "d1b65fc7d…
$ seller_zip_code_prefix <int> 13023, 13844, 20031, 4195, 12914, 20920, 55325…
$ seller_city            <chr> "campinas", "mogi guacu", "rio de janeiro", "s…
$ seller_state           <chr> "SP", "SP", "RJ", "SP", "SP", "RJ", "PE", "SP"…


seller_state,n
<chr>,<int>
SP,1849
PR,349
MG,244
SC,190
RJ,171
RS,129


In [16]:
sellers %>% 
  count(seller_state, sort = T) %>% 
  head()

seller_state,n
<chr>,<int>
SP,1849
PR,349
MG,244
SC,190
RJ,171
RS,129


# Análise de basket

In [27]:
dados_basket <- orders %>% 
  left_join(items, by = "order_id") %>% 
  left_join(products, by = "product_id") %>% 
  as_tibble() %>% 
  distinct(order_id, product_category_name) %>% 
  filter(product_category_name != "") %>% 
  group_by(order_id) %>% 
  filter(n() > 1) %>% 
  ungroup() %>% 
  data.table::as.data.table()

In [28]:
dados_basket %>% head()

order_id,product_category_name
<chr>,<chr>
688052146432ef8253587b930b01a06d,informatica_acessorios
688052146432ef8253587b930b01a06d,malas_acessorios
f2fa3795760c35b8c05fa8e2f09d5143,cama_mesa_banho
f2fa3795760c35b8c05fa8e2f09d5143,moveis_escritorio
f4471dae8c482f51aa1826cd9f5d4433,esporte_lazer
f4471dae8c482f51aa1826cd9f5d4433,beleza_saude


In [9]:
# Transformando em lista por transações, para realizar basket analysis

lista_transacoes <- split(dados_basket[["product_category_name"]], dados_basket[["order_id"]])

# require(arules)

lista_transacoes <- lista_transacoes %>% 
    as("transactions")

# Minerando regras de associação 

regras <- apriori(lista_transacoes, parameter = list(support = 0.0005, confidence = 0.025,
                                          minlen = 2, maxlen = 2))



regras

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
      0.025    0.1    1 none FALSE            TRUE       5   5e-04      2
 maxlen target   ext
      2  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 0 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[61 item(s), 727 transaction(s)] done [0.00s].
sorting and recoding items ... [61 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2

Warning message in apriori(lista_transacoes, parameter = list(support = 5e-04, confidence = 0.025, :
“Mining stopped (maxlen reached). Only patterns up to a length of 2 returned!”

 done [0.00s].
writing ... [355 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


set of 355 rules 

In [23]:
require(reticulate)
use_python("/opt/anaconda3/bin/python3")

In [40]:

aprioripy <- reticulate::import("mlxtend.preprocessing")
pandas <- reticulate::import("pandas")

In [26]:
te = aprioripy$TransactionEncoder()

In [64]:
dataset = pandas$get_dummies(dados_basket$product_category_name)
dataset = r_to_py(dataset)
#dataset = pandas$DataFrame(dataset)

In [71]:
tefit = te$fit(dataset)
tefit_transf = tefit$transform(dataset)

In [79]:
df = pandas$DataFrame(dataset)

In [114]:
apriorip = reticulate::import('mlxtend.frequent_patterns')

In [115]:
apriorip$apriori(df, use_colnames = 'True', min_support = 0.00001,  verbose = 1, max_len = 2)

support,itemsets
<dbl>,<list>
0.0054458816,<environment: 0x55fa33ee8680>
0.0027229408,<environment: 0x55fa33df4468>
0.0040844112,<environment: 0x55fa33d7a680>
0.0013614704,<environment: 0x55fa33d403c8>
0.0006807352,<environment: 0x55fa33cb6f98>
0.0006807352,<environment: 0x55fa33c98aa0>
0.0040844112,<environment: 0x55fa33c8ad20>
0.0238257318,<environment: 0x55fa33c79f10>
0.0633083730,<environment: 0x55fa33c406e0>


In [28]:
redundant <- which(colSums(is.subset(regras, regras)) > 1)
regras <- regras[-redundant]

In [29]:
regras

set of 109 rules 

In [30]:

# 
basket <- regras %>% 
  as("data.frame")

basket <- basket %>% 
  tidyr::separate(rules, into = c("Antecedente", "Consequente"), sep = "=>") %>% 
  mutate_at(vars(1:2), function(x){stringr::str_remove_all(x, "\\{") %>% 
      stringr::str_remove_all("\\}")}) %>% 
  arrange(-lift, -support, -confidence) %>%
  as_tibble()

basket  %>% 
  head()


Antecedente,Consequente,support,confidence,lift,count
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
pc_gamer,cool_stuff,0.001375516,1.0,11.015152,1
fashion_calcados,bebes,0.002751032,1.0,7.817204,2
sinalizacao_e_seguranca,bebes,0.001375516,1.0,7.817204,1
artes_e_artesanato,brinquedos,0.001375516,0.5,7.270000,1
dvds_blu_ray,informatica_acessorios,0.001375516,0.5,7.127451,1
climatizacao,informatica_acessorios,0.001375516,0.5,7.127451,1
